In [52]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
import matplotlib.pyplot as plt

In [53]:
path = os.getcwd()
mixer.init()
Alarm= mixer.Sound('beep_alarm.wav')

try:
    face = cv2.CascadeClassifier('haar-cascade-files/haarcascade_frontalface_alt.xml')
    lefteyeCascade = cv2.CascadeClassifier('haar-cascade-files/haarcascade_lefteye_2splits.xml')
    righteyeCascade = cv2.CascadeClassifier('haar-cascade-files/haarcascade_righteye_2splits.xml')
#     eyePair_cascade = cv2.CascadeClassifier('./haar-cascade/haarcascades_haarcascade_mcs_eyepair_big.xml') #eye_pair
    print("Cascade loaded successfully")
    
except:
    print("Can not open file")


try:
    model = load_model('my_modelexp1.h5')
    print("Model loaded successfully")
except:
    print("Model Initialization Failed")
    
right_eye_predictions=[1]
left_eye_predictions=[1]

Cascade loaded successfully
Model loaded successfully


In [54]:
label=['Close','Open']
total=0


try:
    cap = cv2.VideoCapture(1)
except:
    
    print("Camera initialization Failed")

while(True):
    try:
        ret, frame = cap.read()
    except:
        print("Unable to read image")
        
    height, width = frame.shape[:2] 

    grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(grayscale,minNeighbors=5,scaleFactor=1.1,minSize=(5,5))
    left_eye_roi = lefteyeCascade.detectMultiScale(grayscale)
    right_eye_roi =  righteyeCascade.detectMultiScale(grayscale)
    
    cv2.rectangle(frame, (0,height-50) ,(200,height) ,(0,0,0) ,thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

        for (x,y,w,h) in right_eye_roi:
            
            r_roi=frame[y:y+h,x:x+w]
            r_roi = cv2.cvtColor(r_roi,cv2.COLOR_BGR2GRAY)
            r_roi = cv2.resize(r_roi,(24,24))
            r_roi= r_roi/255
            r_roi=  r_roi.reshape(24,24,-1)
            r_roi = np.expand_dims(r_roi,axis=0)
            
            right_eye_predictions = model.predict_classes(r_roi)
            
            if(right_eye_predictions[0]==0):
                label='Open' 
            
            if(right_eye_predictions[0]==1):
                label='Closed'
            
            break

        for (x,y,w,h) in left_eye_roi:
           
            l_roi=frame[y:y+h,x:x+w]
            l_roi = cv2.cvtColor(l_roi,cv2.COLOR_BGR2GRAY)  
            l_roi = cv2.resize(l_roi,(24,24))
            l_roi= l_roi/255
            l_roi=l_roi.reshape(24,24,-1)
            l_roi = np.expand_dims(l_roi,axis=0)
            
            left_eye_predictions = model.predict_classes(l_roi)
            
            if(left_eye_predictions[0]==0):
                label='Open'   
            if(left_eye_predictions[0]==1):
                label='Closed'
            
            break
    
    font = cv2.FONT_HERSHEY_COMPLEX
    
    if(right_eye_predictions[0]==0 and left_eye_predictions[0]==0):
        
        total=total+1
        cv2.putText(frame,"Eyes: Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    else:
        total=total-1
        cv2.putText(frame,"Eyes: Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(total<0):
        
        total=0   
    
    cv2.putText(frame,'Closed Count:'+str(total),(400, height-20 ),font , 1,(0,0,255),1,cv2.LINE_AA)
    
    if(total>15):
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        
        try:
            Alarm.play()
        
        except:  
            pass
        
    cv2.imshow('frame',frame)
    
    """
    Press q to escape
   
    """
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
